## Load dataset

In [ ]:
import numpy as np
import pandas as pd
import pickle

def get_t(d, time_id):
    # get first time_bin after a publication
    if pd.isna(d):
        return None
    for k, v in time_id.items():
        if d < v:
            return k

        


df_authority = pd.read_parquet("data/authority_file_cac_alma.parquet")
df_authority = df_authority.drop_duplicates(subset="final_id")
CHExNet = pickle.load(open("data/CHExNet.pkl", 'rb'))
time_id = pickle.load(open("data/time_id_final.pkl", 'rb'))
m_AUJ = CHExNet['layer_1']
m_BJ = CHExNet['layer_2']

# get first time_bin after a publication
df_authority["tid_from"] = df_authority.first_polish_pub.apply(lambda x: get_t(x, time_id))




## Calculate exporusre per time bin

In [ ]:
from datetime import date

# Per-time-slice features for each layer
dfs_auj, dfs_bj = [], []

# Union of time keys across AUJ and BJ (unordered; sort(ts) if needed)
ts = list(set(m_AUJ.keys()) | set(m_BJ.keys()))

for t in ts:

    # --- AUJ ---
    auj_v = m_AUJ.get(t)
    if auj_v:
        W = auj_v["matrix"]                 # sparse adjacency/exposure matrix
        ids = list(auj_v["ids_pos_mat"])    # node ids in matrix row/col order
        cur_time = auj_v["time"]            # snapshot date

        deg = np.asarray(W.getnnz(axis=1)).ravel()  # row nnz = degree proxy

        # adopter indicator: 1 if first_polish_pub < cur_time else 0
        z = (
            df_authority[df_authority.final_id.isin(ids)]
            .first_polish_pub
            .apply(lambda x: x < cur_time if isinstance(x, date) else False)
            .to_numpy()
            .astype("int8")
        )

        x = W @ z                            # exposure to adopters (neighbor sum)
        dfs_auj.append(pd.DataFrame({"person_id": ids, "deg1": deg, "x1": x, "t": t,
                                     "combined": [(i, t) for i in ids]}))

    # --- BJ ---
    bj_v = m_BJ.get(t)
    if bj_v:
        W = bj_v["matrix"]
        ids = list(bj_v["ids_pos_mat"])
        cur_time = bj_v["time"]

        deg = np.asarray(W.getnnz(axis=1)).ravel()
        z = (
            df_authority[df_authority.final_id.isin(ids)]
            .first_polish_pub
            .apply(lambda x: x < cur_time if isinstance(x, date) else False)
            .to_numpy()
            .astype("int8")
        )

        x = W @ z
        dfs_bj.append(pd.DataFrame({"person_id": ids, "deg2": deg, "x2": x, "t": t,
                                    "combined": [(i, t) for i in ids]}))


In [ ]:
df_a = pd.concat(dfs_auj, ignore_index=True)   # AUJ features
df_b = pd.concat(dfs_bj,  ignore_index=True)   # BJ features

df_expo = df_a.merge(df_b, how="outer", on=["person_id", "t"])  # join layers
df_expo = df_expo.fillna(0).copy()                              # missing -> 0

df_expo = df_expo.merge(                                       # add adoption timing
    df_authority[["final_id", "first_polish_pub", "tid_from"]],
    left_on="person_id", right_on="final_id", how="left"
)

df_expo["y"] = (df_expo["t"] == df_expo["tid_from"]).astype(int)  # event at tid_from
df_py = df_expo.loc[df_expo["tid_from"].isna() | (df_expo["t"] <= df_expo["tid_from"])].copy()  # risk set, exclude pairs after adoption

#calculate century and filter 16, 17 and 18
df_py["century"] = df_py.t.apply( lambda t: time_id[t].year//100 +1)
df_py = df_py[(df_py["century"]>15) & (df_py["century"]<19)]

df_py["any1"] = (df_py["x1"] > 0).astype(int)                         # any AUJ exposure
df_py["any2"] = (df_py["x2"] > 0).astype(int)                         # any BJ exposure
df_py["any_or"] = (df_py["any1"].eq(1) | df_py["any2"].eq(1)).astype(int)  # any exposure (either)


## Generate table for exposure-count sparsity and any-exposure prevalence by layer


In [ ]:
def exposure_row(x: pd.Series) -> dict:
    """Percent in {0,1,>=2} and percent exposed (>=1)."""
    s = x.fillna(0)
    n = len(s)
    return {
        "% x=0":   100.0 * (s.eq(0).sum() / n),
        "% x=1":   100.0 * (s.eq(1).sum() / n),
        "% x>=2":  100.0 * ((s.ge(2)).sum() / n),
        "% exposed": 100.0 * ((s.ge(1)).sum() / n),
    }

# layer rows
tab = pd.DataFrame.from_dict(
    {
        "Layer 1": exposure_row(df_py["x1"]),
        "Layer 2": exposure_row(df_py["x2"]),
    },
    orient="index",
)

# either / both (any exposure indicators)
any1 = df_py["x1"].fillna(0).gt(0)
any2 = df_py["x2"].fillna(0).gt(0)

tab.loc["Either layer", ["% exposed"]] = 100.0 * (any1 | any2).mean()
tab.loc["Both layers",  ["% exposed"]] = 100.0 * (any1 & any2).mean()

tab = tab[["% x=0", "% x=1", "% x>=2", "% exposed"]].round(2)

tab.to_csv("exposure_count_sparsity.csv")

## Fit contagion models and create results table

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
from firthmodels.adapters.statsmodels import FirthLogit

# -------------------------
# 1) Specify models
# -------------------------
models = {
    "M0": "y ~ C(century)",
    "M1": "y ~ C(century) + any1",
    "M2": "y ~ C(century) + any2",
    "M3": "y ~ C(century) + any1 + any2",
    "M4": "y ~ C(century) + any_or",
    "M5": "y ~ C(century) + any1 + any2 + any1:any2",
}
key_terms = ["any1", "any2", "any_or", "any1:any2"]

# -------------------------
# 2) Fit Firth models
# -------------------------
res = {name: FirthLogit.from_formula(formula, df_py).fit()
       for name, formula in models.items()}

def safe_get(obj, attr, default=np.nan):
    v = getattr(obj, attr, None)
    return default if v is None else float(v)

# -------------------------
# 3) Manual AIC/BIC + LR helper
# -------------------------
def manual_aic_bic(llf, k, n):
    # AIC = -2*llf + 2*k ; BIC = -2*llf + log(n)*k 
    aic = -2*llf + 2*k
    bic = -2*llf + np.log(n)*k
    return float(aic), float(bic)

def lr_test(full_res, base_res):
    if not (hasattr(full_res, "llf") and hasattr(base_res, "llf")):
        return (np.nan, 0, np.nan)
    lr_stat = 2 * (float(full_res.llf) - float(base_res.llf))
    df_diff = int(len(np.asarray(full_res.params)) - len(np.asarray(base_res.params)))
    pval = float(chi2.sf(lr_stat, df_diff))
    return float(lr_stat), df_diff, pval

# -------------------------
# 4) Build Table 4 with OR [95% CI] and manually-computed AIC/BIC
# -------------------------
m0 = res["M0"]
rows = []

for name, r in res.items():
    # names for parameters (adapter should have exog_names)
    if hasattr(r, "model") and hasattr(r.model, "exog_names"):
        names = list(r.model.exog_names)
    else:
        names = [f"x{i}" for i in range(len(np.asarray(r.params)))]

    params = pd.Series(np.asarray(r.params), index=names, name="beta")
    p_two  = pd.Series(np.asarray(getattr(r, "pvalues", np.full(len(names), np.nan))), index=names, name="p_two")

    ci_arr = np.asarray(r.conf_int())
    ci_b = pd.DataFrame(ci_arr, index=names, columns=["beta_low", "beta_high"])

    llf = safe_get(r, "llf")
    k = int(len(params))  # number of estimated parameters (incl intercept + dummies)
    n = int(getattr(r, "nobs", len(df_py)))

    AIC = np.nan
    BIC = np.nan
    if np.isfinite(llf):
        AIC, BIC = manual_aic_bic(llf, k, n)

    row = {
        "model": name,
        "formula": models[name],
        "llf": llf,
        "nobs": n,
        "k_params": k,
        "AIC": AIC,
        "BIC": BIC,
    }

    for term in key_terms:
        col_prefix = f"exp(beta_{term})"
        if term in params.index:
            b = float(params[term])
            OR = float(np.exp(b))
            OR_low = float(np.exp(ci_b.loc[term, "beta_low"]))
            OR_high = float(np.exp(ci_b.loc[term, "beta_high"]))
            row[f"{col_prefix}_CI"] = f"{OR:.2f} [{OR_low:.2f}, {OR_high:.2f}]"
            row[f"p({term})"] = float(p_two.get(term, np.nan))
        else:
            row[f"{col_prefix}_CI"] = ""
            row[f"p({term})"] = np.nan

    # LR vs M0 (if llf exists)
    if name != "M0":
        lr, df_diff, p_lr = lr_test(r, m0)
        row["LR_vs_M0"] = lr
        row["df_diff_vs_M0"] = df_diff
        row["p_LR_vs_M0"] = p_lr

        # deltas vs M0 (only if AIC/BIC finite)
        row["AIC_delta_vs_M0"] = (AIC - manual_aic_bic(safe_get(m0, "llf"), len(np.asarray(m0.params)), int(getattr(m0, "nobs", len(df_py))))[0]
                                  if np.isfinite(AIC) and np.isfinite(safe_get(m0, "llf")) else np.nan)
        row["BIC_delta_vs_M0"] = (BIC - manual_aic_bic(safe_get(m0, "llf"), len(np.asarray(m0.params)), int(getattr(m0, "nobs", len(df_py))))[1]
                                  if np.isfinite(BIC) and np.isfinite(safe_get(m0, "llf")) else np.nan)
    else:
        row["LR_vs_M0"] = np.nan
        row["df_diff_vs_M0"] = 0
        row["p_LR_vs_M0"] = np.nan
        row["AIC_delta_vs_M0"] = 0.0
        row["BIC_delta_vs_M0"] = 0.0

    rows.append(row)

table4 = pd.DataFrame(rows).set_index("model")

# -------------------------
# 5) Pretty headers and final display (2 decimals)
# -------------------------
pretty = {
    "any1": r"$\exp(\hat\beta_1)$",
    "any2": r"$\exp(\hat\beta_2)$",
    "any1:any2": r"$\exp(\hat\beta_3)$",
     "any_or": r"$\exp(\hat\beta_or)$",
    
}
rename_map = {}
for term in key_terms:
    rename_map[f"exp(beta_{term})_CI"] = pretty[term] + " [95% CI]"
    rename_map[f"p({term})"] = f"p({pretty[term]})"
table4 = table4.rename(columns=rename_map)

# -------------------------
# 6) Final display (2 decimals)  -- now includes any_or
# -------------------------
table4_display = table4[[
    "formula", "llf", "AIC", "BIC",

    pretty["any1"] + " [95% CI]", f"p({pretty['any1']})",
    pretty["any2"] + " [95% CI]", f"p({pretty['any2']})",
    pretty["any_or"] + " [95% CI]", f"p({pretty['any_or']})",          # <-- add
    pretty["any1:any2"] + " [95% CI]", f"p({pretty['any1:any2']})",

    "LR_vs_M0", "df_diff_vs_M0", "p_LR_vs_M0", "AIC_delta_vs_M0", "BIC_delta_vs_M0",
]].copy()

# round numeric model-fit columns to 2 decimals
num_cols = ["llf", "AIC", "BIC", "LR_vs_M0", "AIC_delta_vs_M0", "BIC_delta_vs_M0"]
table4_display[num_cols] = table4_display[num_cols].round(2)

# format p-values to 2 decimals
p_cols = [
    f"p({pretty['any1']})",
    f"p({pretty['any2']})",
    f"p({pretty['any_or']})",            
    f"p({pretty['any1:any2']})",
    "p_LR_vs_M0"
]
table4_display[p_cols] = table4_display[p_cols].applymap(
    lambda x: "" if pd.isna(x) else f"{float(x):.2f}"
)

table4_display.to_csv("table_contagion_firth.csv")


## Generate crude hazard by decade plot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

d = df_py.copy()

# map t -> calendar year
d["year"] = d["t"].map(lambda t: time_id[t].year)

# decade start (1503 -> 1500, 1519 -> 1510, etc.)
d["decade"] = (d["year"] // 10) * 10

life_dec = (
    d.groupby("decade", as_index=False)
     .agg(at_risk=("person_id", "count"),
          events=("y", "sum"))
     .sort_values("decade")
)

life_dec["hazard"] = life_dec["events"] / life_dec["at_risk"]

# ---- plot ----
x = life_dec["decade"] + 5  # decade midpoint

fig, ax_h = plt.subplots(figsize=(11, 4))

# line = hazard (left axis)
ax_h.plot(x, life_dec["hazard"], color="black", linewidth=2)
ax_h.set_xlabel("Decade")
ax_h.set_ylabel("Crude hazard (events / at risk)")

# bars = counts (right axis)
ax_e = ax_h.twinx()  # shares x-axis; ticks on the right 
ax_e.bar(x, life_dec["events"], width=8, color="steelblue", alpha=0.25, edgecolor="none")
ax_e.set_ylabel("First adoptions (count)")

# right axis: integers only (no fractions)
ax_e.yaxis.set_major_locator(MaxNLocator(integer=True, min_n_ticks=1))  
ax_e.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))             

# paper-friendly styling
ax_h.set_xlim(life_dec["decade"].min(), life_dec["decade"].max() + 10)
ax_h.spines["top"].set_visible(False)
ax_e.spines["top"].set_visible(False)

ax_h.set_title("Crude hazard by decade (with first-adoption counts)")
plt.tight_layout()
plt.savefig("crude_hazard.png", dpi=300)
plt.show()
